# Inundation Dynamics Simulation 

## Create an ArcGIS Pro project

Open ArcGIS Pro and create a new project titled `inundation`.

## Clone the arcgispro-py3 env

Clone the `arcgispro-py` env to create a new env named `arcgispro-py3-clone`.

![image](https://github.com/user-attachments/assets/52836e4a-7246-48d2-8081-27ec6f21395f)

## Install scikit-image 

Activate the `arcgispro-py3-clone` env and install the `scikit-image` package into the env.

![image](https://github.com/user-attachments/assets/320c21a1-7ad1-4df7-87c4-800945a1edc1)

## Import libraries

In [ ]:
import os
import arcpy

## Set workspace

Set to working space to the project folder instead of a GeoDatabase.

In [ ]:
arcpy.env.workspace = os.path.dirname(arcpy.env.workspace)
print(arcpy.env.workspace)

## Download the lidar toolbox

Click this [link](https://github.com/opengeos/lidar/archive/refs/heads/master.zip) and download it to the inundation project folder. Unzip the downloaded file and rename the folder from `lidar-master` to `lidar.`

You will find the ArcGIS toolbox under `inundation\lidar\lidar\toolbox\ArcGIS Pro Hydrology Analyst.tbx`

## Import the lidar toolbox

In [ ]:
# Path to the custom toolbox
toolbox_path = r"lidar\lidar\toolbox\ArcGIS Pro Hydrology Analyst.tbx"  # Change to your toolbox path

# Import the toolbox
arcpy.ImportToolbox(toolbox_path, "HydroTools")

## Set input data and output folder

In [ ]:
input_dem = os.path.join(arcpy.env.workspace, r"lidar\examples\lidar-dem\dem_full.tif")
out_dir = os.path.join(os.path.expanduser("~\Downloads"), "output")

In [ ]:
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

In [ ]:
print(input_dem)

In [ ]:
print(out_dir)

In [ ]:
arcpy.env.workspace = out_dir

## Extract sinks

In [ ]:
arcpy.HydroTools.ExtrackSink(
    Input_DEM=input_dem,
    Minimum_Sink_Size=1000,
    Minimum_Sink_Depth__from_water_surface_to_spill_point_=1,
    Buffer_Distance=0,
    Output_Sink_Polygon="sink.shp",
)

## Delineate catchments

In [ ]:
arcpy.HydroTools.DelineateCatchment(
    Input_Partially_Filled_DEM="dem_partially_filled.tif",
    Input_Sink_Polygon="sink.shp",
    Output_Catchment_Polygon="catchment.shp",
)

## Delineate flowpaths

In [ ]:
arcpy.HydroTools.DelineateFlowPath(
    Input_Fully_Filled_DEM="dem_fully_filled.tif",
    Input_Sink_Polygon="sink.shp",
    Rainfall_Intensity__cm_h_="5",
    Output_Flow_Path="flowpath.shp",
)

## Delineate depression hierarchy

In [ ]:
arcpy.HydroTools.DelineateDepressionHierarchy(
    Input_DEM_Sink="sink.tif",
    Minimum_Depression_Size="1000",
    Minimum_Depression_Depth="0.5",
    Slicing_Interval="0.2",
    Output_Depression_Level_Image="level.tif",
)

## Delinate catchment hierarchy

In [ ]:
arcpy.HydroTools.CatchmentHierarchy(
    Input_Partially_Filled_DEM="dem_partially_filled.tif",
    Input_Depression_Hierarchy_Shapefiles="shp",
    Output_Catchment_Hierarchy="catchment_hir.tif",
)

## Simulate inundation

In [ ]:
os.makedirs(os.path.join(arcpy.env.workspace, "simulation"), exist_ok=True)

In [ ]:
arcpy.HydroTools.SimulateInundation(
    Input_Sink_Image="sink.tif",
    Input_Catchment_Hierarchy_Image="catchment_hir.tif",
    Minimum_Depression_Size="1000",
    Minimum_Depression_Depth="0.2",
    Slicing_Interval="0.2",
    Rainfall_Intensity__cm_h_="5",
    Rainfall_Duration__h_="50",
    Simulation_Time_Step__h_="1",
    Output_Inundation_Image_Folder="simulation",
)

## Play the animation

In [ ]:
arcpy.HydroTools.PlayAnimation(
    Input_DEM="dem_partially_filled.tif",
    Loops="3",
    Input_Inundation_Image_Folder="simulation",
)